# Multi-Agent Insight Grok Creator (Semantic Kernel + Ollama)

This notebook explores a multi-agent workflow where your saved insights are expanded into **groks**—concise notes that deepen understanding.

## 🧠 Workflow Overview
1. **Input** an insight and its learning source
2. **Agent 1** analyzes the insight and recommends how to create groks
3. **Specialized Agents** craft contradiction, metaphor, or simplification groks based on the plan
4. **Output** is a structured set of groks ready for your learning vault

## 🤖 Agents
- **Planner Agent** – Chooses whether a contradiction, metaphor, or simplification will best expand the insight
- **Contradiction Agent** – Crafts critical thinking groks when contradictions are requested
- **Metaphor Agent** – Builds relatable metaphors and analogies
- **Simplification Agent** – Distills the insight to its essence with plain-language groks

We'll use Semantic Kernel with an Ollama-served model to coordinate these agents and produce rich learning artifacts.

In [1]:
# Import required libraries
import sys
import asyncio
import json
import re
import textwrap
from typing import Any, Dict, List

sys.path.append('..')

import semantic_kernel as sk
from semantic_kernel.connectors.ai.ollama import OllamaChatCompletion
from semantic_kernel.contents import ChatHistory
from semantic_kernel.functions import kernel_function
from semantic_kernel.functions.kernel_arguments import KernelArguments
from semantic_kernel.connectors.ai.prompt_execution_settings import PromptExecutionSettings

from config import config

print('🧭 Multi-Agent Grok Creator Configuration:')
print(f'  Ollama URL: {config.ollama_base_url}')
print(f'  Model: {config.model_name}')
print(f'  Temperature: {config.temperature}')

🧭 Multi-Agent Grok Creator Configuration:
  Ollama URL: http://192.168.1.82:11434
  Model: qwen3:14b
  Temperature: 0.7


In [2]:
# Initialize Semantic Kernel with Ollama
kernel = sk.Kernel()

service_id = 'ollama_groks'
ollama_service = OllamaChatCompletion(
    ai_model_id=config.model_name,
    host=config.ollama_base_url,
    service_id=service_id
)

kernel.add_service(ollama_service)

print('✅ Semantic Kernel connected to Ollama!')
print(f'Service ID: {service_id}')

✅ Semantic Kernel connected to Ollama!
Service ID: ollama_groks


In [3]:
# Insight Grok Multi-Agent Plugin
class InsightGrokPlugin:
    """Plugin hosting the planner agent and the specialized grok creators."""

    async def _call_model(self, prompt: str, max_tokens: int = 900, temperature: float = 0.7) -> str:
        chat_service = kernel.get_service(type=OllamaChatCompletion)
        chat_history = ChatHistory()
        chat_history.add_user_message(prompt)

        settings = PromptExecutionSettings(
            max_tokens=max_tokens,
            temperature=temperature
        )

        response = await chat_service.get_chat_message_contents(
            chat_history=chat_history,
            settings=settings
        )

        return response[0].content

    @kernel_function(
        description='Analyze an insight and recommend which groks to create',
        name='analyze_insight'
    )
    async def analyze_insight(self, insight: str, source: str = '') -> str:
        template = textwrap.dedent("""
            You are the lead planner inside a multi-agent knowledge team.

            Your job is to study the learner's insight and decide which learning groks to create.

            Respond ONLY in strict JSON with this schema:
            {{
              "decision_summary": str,
              "reasoning": [str, ...],
              "recommended_groks": [
                {{
                  "type": one of ["contradiction", "metaphor", "simplification"],
                  "title": str,
                  "goal": str,
                  "focus_points": [str, ...],
                  "priority": one of ["high", "medium", "low"],
                  "notes_for_agent": str
                }}
              ],
              "follow_up_questions": [str, ...]
            }}

            Insight: {insight}
            Source: {source}

            Always include at least one recommended_grok.
            If none of the specialized agents are useful, choose simplification.
        """)
        source_text = source if source else 'Unknown'
        planner_prompt = template.format(insight=insight, source=source_text)

        return await self._call_model(planner_prompt, max_tokens=700, temperature=0.4)

    @kernel_function(
        description='Create contradiction-focused groks',
        name='generate_contradiction_groks'
    )
    async def generate_contradiction_groks(self, insight: str, source: str = '', plan: str = '') -> str:
        template = textwrap.dedent("""
            You are the Contradiction Agent. Use the planner guidance below to craft critical-thinking groks.

            Planner guidance: {plan_context}

            Insight: {insight}
            Source: {source}

            Produce JSON with schema:
            {{
              "groks": [
                {{
                  "title": str,
                  "contradiction": str,
                  "supporting_reasoning": [str, ...],
                  "reflection_questions": [str, ...]
                }}
              ],
              "agent_notes": str
            }}

            Create 1-2 contradiction groks. Each contradiction must be respectful and sharpen critical thinking.
        """)
        source_text = source if source else 'Unknown'
        plan_context = plan if plan else 'None provided.'
        contradiction_prompt = template.format(
            plan_context=plan_context,
            insight=insight,
            source=source_text
        )

        return await self._call_model(contradiction_prompt, max_tokens=650, temperature=0.6)

    @kernel_function(
        description='Create metaphor-based groks',
        name='generate_metaphor_groks'
    )
    async def generate_metaphor_groks(self, insight: str, source: str = '', plan: str = '') -> str:
        template = textwrap.dedent("""
            You are the Metaphor Agent. Translate the insight into relatable comparisons.

            Planner guidance: {plan_context}

            Insight: {insight}
            Source: {source}

            Return JSON with schema:
            {{
              "groks": [
                {{
                  "title": str,
                  "metaphor": str,
                  "explanation": str,
                  "transferable_lesson": str
                }}
              ],
              "visual_prompt": str,
              "agent_notes": str
            }}

            Generate 1-2 vivid metaphors that make the insight memorable.
        """)
        source_text = source if source else 'Unknown'
        plan_context = plan if plan else 'None provided.'
        metaphor_prompt = template.format(
            plan_context=plan_context,
            insight=insight,
            source=source_text
        )

        return await self._call_model(metaphor_prompt, max_tokens=650, temperature=0.65)

    @kernel_function(
        description='Create simplification groks',
        name='generate_simplification_groks'
    )
    async def generate_simplification_groks(self, insight: str, source: str = '', plan: str = '') -> str:
        template = textwrap.dedent("""
            You are the Simplification Agent. Break the insight into easy-to-digest groks.

            Planner guidance: {plan_context}

            Insight: {insight}
            Source: {source}

            Return JSON with schema:
            {{
              "groks": [
                {{
                  "title": str,
                  "explanation": str,
                  "key_steps": [str, ...],
                  "real_world_hook": str
                }}
              ],
              "practice_prompt": str,
              "agent_notes": str
            }}

            Create 1-2 simplification groks that anyone new to the topic could follow.
        """)
        source_text = source if source else 'Unknown'
        plan_context = plan if plan else 'None provided.'
        simplification_prompt = template.format(
            plan_context=plan_context,
            insight=insight,
            source=source_text
        )

        return await self._call_model(simplification_prompt, max_tokens=650, temperature=0.5)

In [4]:
# Register the plugin and define orchestration helpers
grok_plugin = InsightGrokPlugin()
kernel.add_plugin(grok_plugin, plugin_name='InsightGrok')

print('🎯 Insight Grok agents registered: analyze_insight, generate_contradiction_groks, generate_metaphor_groks, generate_simplification_groks')

def safe_json_loads(payload: str) -> Any:
    """Attempt to parse JSON while preserving the raw text on failure."""
    if not payload:
        return {'raw_text': ''}

    text = payload.strip()
    text = re.sub(r'<think>.*?</think>', '', text, flags=re.DOTALL).strip()

    if text.startswith('```') and text.endswith('```'):
        fenced_lines = [line for line in text.splitlines() if not line.strip().startswith('```')]
        text = '\n'.join(fenced_lines).strip()

    try:
        return json.loads(text)
    except json.JSONDecodeError:
        match = re.search(r'\{.*\}', text, flags=re.DOTALL)
        if match:
            candidate = match.group(0)
            try:
                return json.loads(candidate)
            except json.JSONDecodeError:
                pass
    return {'raw_text': payload}

async def run_multi_agent_pipeline(insight: str, source: str = '') -> Dict[str, Any]:
    planner_fn = kernel.get_function('InsightGrok', 'analyze_insight')
    planner_result = await planner_fn.invoke(
        kernel,
        KernelArguments(insight=insight, source=source)
    )
    planner_text = planner_result.value.strip() if hasattr(planner_result, 'value') else str(planner_result)
    planner_plan = safe_json_loads(planner_text)

    recommendations = []
    if isinstance(planner_plan, dict) and 'recommended_groks' in planner_plan:
        recommendations = planner_plan.get('recommended_groks', [])

    outputs: Dict[str, List[Any]] = {
        'contradiction': [],
        'metaphor': [],
        'simplification': []
    }

    for rec in recommendations:
        if not isinstance(rec, dict):
            continue
        rec_type = rec.get('type', '').lower()
        plan_json = json.dumps(rec, ensure_ascii=False)

        if rec_type == 'contradiction':
            fn = kernel.get_function('InsightGrok', 'generate_contradiction_groks')
        elif rec_type == 'metaphor':
            fn = kernel.get_function('InsightGrok', 'generate_metaphor_groks')
        elif rec_type == 'simplification':
            fn = kernel.get_function('InsightGrok', 'generate_simplification_groks')
        else:
            continue

        agent_result = await fn.invoke(
            kernel,
            KernelArguments(insight=insight, source=source, plan=plan_json)
        )
        agent_text = agent_result.value.strip() if hasattr(agent_result, 'value') else str(agent_result)
        parsed = safe_json_loads(agent_text)
        outputs.setdefault(rec_type, []).append(parsed)

    return {
        'insight': insight,
        'source': source,
        'planner_raw': planner_text,
        'planner_plan': planner_plan,
        'groks': outputs
    }

🎯 Insight Grok agents registered: analyze_insight, generate_contradiction_groks, generate_metaphor_groks, generate_simplification_groks


In [ ]:
# Run the pipeline with your own insight and source
async def display_groks(insight: str, source: str = '') -> None:
    print('🧠 Running multi-agent insight pipeline...')
    result = await run_multi_agent_pipeline(insight, source)

    print('🧠 Planner Decision:')
    print(json.dumps(result.get('planner_plan', {}), indent=2, ensure_ascii=False))

    print('🗂️ Generated Groks:')
    for grok_type, payloads in result.get('groks', {}).items():
        if not payloads:
            continue
        print(f'=== {grok_type.upper()} GROKS ===')
        for idx, payload in enumerate(payloads, 1):
            print(f'[{idx}]')
            print(json.dumps(payload, indent=2, ensure_ascii=False))

    if all(not v for v in result.get('groks', {}).values()):
        print('⚠️ No specialized groks were produced. Review the planner output above.')

# Example values — replace these with your own when exploring.
demo_insight = "Not all double attacks are forks but all forks are double attacks."
demo_source = ("In Chess, a Double Attack is when two simultaneous threats are created after a move. "
               "If the same piece attacks two enemy pieces, it is called a fork.")

await display_groks(demo_insight, demo_source)

2025-10-03 21:21:04,249 - semantic_kernel.functions.kernel_function - INFO - Function InsightGrok-analyze_insight invoking.


♟️ Running multi-agent insight pipeline...


2025-10-03 21:23:45,754 - httpx - INFO - HTTP Request: POST http://192.168.1.82:11434/api/chat "HTTP/1.1 200 OK"
2025-10-03 21:23:45,791 - semantic_kernel.functions.kernel_function - INFO - Function InsightGrok-analyze_insight succeeded.
2025-10-03 21:23:45,793 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 162.187044s
2025-10-03 21:23:45,795 - semantic_kernel.functions.kernel_function - INFO - Function InsightGrok-generate_simplification_groks invoking.
2025-10-03 21:23:45,791 - semantic_kernel.functions.kernel_function - INFO - Function InsightGrok-analyze_insight succeeded.
2025-10-03 21:23:45,793 - semantic_kernel.functions.kernel_function - INFO - Function completed. Duration: 162.187044s
2025-10-03 21:23:45,795 - semantic_kernel.functions.kernel_function - INFO - Function InsightGrok-generate_simplification_groks invoking.
2025-10-03 21:25:28,271 - httpx - INFO - HTTP Request: POST http://192.168.1.82:11434/api/chat "HTTP/1.1 200 OK"
2025-10-03

🧠 Planner Decision:
{
  "decision_summary": "The insight establishes a hierarchical relationship between double attacks and forks in chess. A simplification grok is essential to clarify this relationship, while a metaphor grok can help visualize the subset structure.",
  "reasoning": [
    "The insight reveals that forks are a specialized subset of double attacks, requiring a clear explanation of their distinction.",
    "A simplification grok will help learners grasp the fundamental difference between the two concepts.",
    "A metaphor grok can make the abstract hierarchy more tangible by comparing it to a parent-child relationship."
  ],
  "recommended_groks": [
    {
      "type": "simplification",
      "title": "Understanding Double Attacks vs. Forks",
      "goal": "Clarify that forks are a specific type of double attack where a single piece creates two threats.",
      "focus_points": [
        "Define double attack as any move creating two simultaneous threats",
        "Expla

In [ ]:
# Optional: helper to run custom insights inside the notebook
async def run_custom_insight(insight: str, source: str = '') -> None:
    await display_groks(insight, source)

print('ℹ️ Use await run_custom_insight("your insight", "source") to generate groks interactively.')

## ✅ Multi-Agent Grok Creator Summary

### What we built
- **Planner Agent** that orchestrates which groks matter most
- **Specialized creators** for contradictions, metaphors, and simplifications
- **Semantic Kernel orchestration** that stitches results into a single structured output

### How to use it
1. Provide an insight and its source
2. Let the planner decide the most valuable groks
3. Collect the generated contradiction/metaphor/simplification groks
4. Save them into your learning vault to reinforce understanding

### Next ideas
- Add a **quiz agent** that turns groks into spaced-repetition prompts
- Track agent choices over time to build a learner profile
- Persist groks to a database or markdown knowledge base automatically

This notebook is ready to power your multi-agent learning workflow!